# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Francisco Joaquin Murcia Gomez  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/13_H_75SPPYNoGVqZOalVjRVPVBVm6B6k?usp=sharing <br>
Problema:
>1. Sesiones de doblaje <br>


Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Los datos son:

- Número de actores: 10
- Número de tomas: 30
....







                                        

# Modelo
### ¿Como represento el espacio de soluciones?

La estructura de datos que ha elegido para representar la solución es una lista de listas, donde cada lista interna representa las tomas asignadas a un día específico de grabación, un ejemplo de solución seria:

```python 
[[0, 10, 11, 25, 24, 3], [5, 19, 6, 9, 21, 2], [4, 12, 7, 8, 1, 14], [13, 28, 15, 16, 17, 18], [20, 22, 23, 26, 27, 29]]
```
donde cada sublista representa un día de grabación, del primer día al quinto día, en cada sublista loa números representan las tomas que se van a rodar ese día, en ejemplo dado, en el día 2 se van a rodar las tomas 5, 19, 6, 9, 21 y 2.

A la hora de imprimir el resultado he decidido que me imprima el orden de las tomas y hacer una tabla donde se represente los días, tomas a rodar, el número de actores convocados y el costo del día:

```
Calendario de sesiones:
Orden de las tomas: [18, 24, 7, 2, 9, 28, 26, 21, 16, 8, 13, 11, 25, 19, 30, 17, 22, 23, 29, 1, 20, 12, 14, 10, 15, 4, 3, 5, 27, 6]
Horario:
Día                  Tomas  Numero de actores  Coste de la sesion (€)
    1    18, 24, 7, 2, 9, 28                  6                     180
    2  26, 21, 16, 8, 13, 11                  9                     270
    3 25, 19, 30, 17, 22, 23                  5                     150
    4  29, 1, 20, 12, 14, 10                  7                     210
    5     15, 4, 3, 5, 27, 6                  6                     180
    
Coste Total:: 990€
```

### ¿Cual es la función objetivo?

La función objetivo sería minimizar el numero ve veces que tengo que convocar a los actores ($A_i$) para que el coste de la producción sea el mínimo.

$$
\text{Minimizar } Z = \sum_{i=1}^{D} A_i
$$

Aquí, $Z$ es el objetivo que buscamos minimizar, que representa el total de actores-día necesarios para completar la grabación, sumando los actores necesarios en cada uno de los $D$ días de grabación. La meta es organizar el calendario de grabación de manera que este total ($Z$) sea lo más bajo posible, lo que indirectamente minimizará el costo total bajo la premisa de que el costo por actor por día es constante y no se considera en la optimización directamente.
### ¿Como implemento las restricciones?

Tendremos que aplicar dos principales restricciones:
- Restricción de tomas por día: Se asegura de que cada lista interna no contenga más de 6 tomas, lo que refleja la limitación de no poder grabar más de 6 tomas por día.
- Asignación de todas las tomas: Se debe asegurar que cada toma esté asignada a algún día.

## Análisis
### ¿Que complejidad tiene el problema? Orden de complejidad y Contabilizar el espacio de soluciones

Sin restricciones, el número total de posibilidades es $\left(\binom{n}{0} + \binom{n}{1} + ... + \binom{n}{n-1} + \binom{n}{n} \right) n! = 2^n n!$  tomas resulta en aproximadamente $2.8 \cdot 10^{41}$.

Con restricciones, considerando solo hasta 6 tomas por día, el número total de posibilidades se calcula como $\left(\binom{n}{0} + \binom{n}{1} + ... + \binom{n}{6} \right) n! $, resulta en aproximadamente $2.04 \cdot 10^{38}$.



# Diseño

In [2]:
import numpy as np
import itertools
import pandas as pd

TABLA_ESCENAS = np.array([
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
    [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
])
N_ESCENAS, N_ACTORES = TABLA_ESCENAS.shape
N_DIAS = (N_ESCENAS + 5) // 6
MAX_TOMAS_POR_DIA = 6
COSTO_POR_ACTOR_POR_DIA = 30
N_INTENTOS_MAX = 10

C:\Users\Usuario\AppData\Local\Temp\ipykernel_12580\1192134680.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def print_schedule(session_order=list(range(N_ESCENAS))):
    schedule = [session_order[n : n + MAX_TOMAS_POR_DIA] for n in range(0, len(session_order), MAX_TOMAS_POR_DIA)]
    schedule_details = []

    print("\n\nCalendario de sesiones:")
    print(f"Orden de sesiones: {[x + 1 for x in session_order]}")

    total_cost = 0
    for day_num, day in enumerate(schedule, start=1):
        shots = [x + 1 for x in day]
        sum_shots = np.sum(TABLA_ESCENAS[day, :], axis=0)
        num_actors = np.count_nonzero(sum_shots)
        cost_per_day = num_actors * COSTO_POR_ACTOR_POR_DIA
        total_cost += cost_per_day

        schedule_details.append(
            {
                "Día": day_num,
                "Tomas": ", ".join(map(str, shots)),
                "Numero de actores": num_actors,
                "Coste de la sesion (€)": cost_per_day,
            }
        )

    df_schedule = pd.DataFrame(schedule_details)

    print("Horario:")
    print(df_schedule.to_string(index=False))

    print(f"\nCoste Total:: {total_cost}€")
def evaluar_solucion(solucion, data):
    coste = 0
    for sesion in solucion:
        # Accede a las filas especificadas por los índices de 'sesion' en el array 'data'
        data_sesion = data[sesion, :]
        # Calcula si hay algún valor no cero en cada columna de 'data_sesion' y suma esos valores
        # np.any(data_sesion != 0, axis=0) devuelve un array booleano a lo largo del eje de las columnas
        # .sum() suma los valores True (considerados como 1)
        coste += np.any(data_sesion != 0, axis=0).sum()

    return coste

### Fuerza bruta

La fuerza bruta es conceptualmente simple y sencillo de implementar. El enfoque de fuerza bruta, aunque garantiza encontrar la solución óptima al evaluar todas las combinaciones posibles de tomas y actores por día, resulta extremadamente ineficiente para este problema grandes debido al vasto número de combinaciones a considerar, lo que lleva a tiempos de ejecución muy largos. A pesar de que las restricciones del problema ayudan a reducir este número, la escala del problema hace que el método siga siendo poco práctico.

In [20]:

BEST_COST = float("inf")  # Global variable to track the best total number of actors
BEST_SCHEDULE = np.array([])


def get_shot_combinations(remaining_shots, n):
    """Genera todas las combinaciones posibles de tomas a partir de las tomas restantes."""
    return np.array(list(itertools.combinations(remaining_shots, n)), dtype=int)


def calculate_actors_in_shots(shots):
    """Calcula el número de actores que participan en un conjunto de tomas."""
    return len(np.unique(TABLA_ESCENAS[shots].nonzero()[1]))


def save_posible_best_schedule(selected_shots, total_actors):
    global BEST_COST
    global BEST_SCHEDULE

    if total_actors < BEST_COST:
        BEST_COST = total_actors
        BEST_SCHEDULE = selected_shots.copy()
        #print_schedule(BEST_SCHEDULE.flatten())
        print(f"Coste: {BEST_COST*COSTO_POR_ACTOR_POR_DIA}€")


def generate_schedule_brute_algorithm(remaining_shots, selected_shots, total_actors):
    """Ejecuta el algoritmo por fuerza bruta"""

    if len(remaining_shots) == 0:
        save_posible_best_schedule(selected_shots, total_actors)
        return

    shot_combinations = get_shot_combinations(remaining_shots, MAX_TOMAS_POR_DIA)

    for shots in shot_combinations:
        actors = calculate_actors_in_shots(shots)
        new_selected_shots = np.vstack((selected_shots, shots))
        new_total_actors = total_actors + actors
        new_remaining_shots = np.array([shot for shot in remaining_shots if shot not in shots])

        generate_schedule_brute_algorithm(new_remaining_shots, new_selected_shots, new_total_actors)

initial_schedule = np.arange(N_ESCENAS)
np.random.shuffle(initial_schedule)
try:
    generate_schedule_brute_algorithm(initial_schedule, np.empty((0, MAX_TOMAS_POR_DIA), dtype=int), 0)
except KeyboardInterrupt:
    print_schedule(BEST_SCHEDULE.flatten())
    #print(evaluar_solucion(BEST_SCHEDULE,TABLA_ESCENAS))

Coste: 1110€
Coste: 1080€
Coste: 1050€
Coste: 1020€
34


## Algoritmo genetico
(por implementar)

## Algoritmo voraz
(por implementar)

In [9]:
import numpy as np
from copy import deepcopy

MAX_TOMAS_POR_DIA = 6


def evaluar_solucion(solucion, data):
    coste = 0
    for sesion in solucion:
        coste += np.any(data[sesion, :] != 0, axis=0).sum()

    return coste


def ordenar_tomas(tomas, data):
    suma_filas = np.sum(data, axis=1)
    indices_ordenados = np.argsort(suma_filas)[::-1]
    return tomas[indices_ordenados]


def seleccion_voraz(solucion, sesion, tomas, data):
    mejor_toma = tomas[0]

    solucion_temp = deepcopy(solucion)
    sesion_temp = deepcopy(sesion)

    solucion_temp.append(sesion_temp + [mejor_toma])
    menor_coste = evaluar_solucion(solucion_temp, data)
    for i in range(1, len(tomas)):
        solucion_temp = deepcopy(solucion)
        sesion_temp = deepcopy(sesion)
        solucion_temp.append(sesion_temp + [tomas[i]])
        coste = evaluar_solucion(solucion_temp, data)

        if coste < menor_coste:
            mejor_toma = tomas[i]
            menor_coste = coste

    return mejor_toma


def busqueda_voraz(data):

    solucion = []
    tomas = np.arange(data.shape[0])
    tomas = ordenar_tomas(tomas, data)

    ses_cont = 0
    while tomas.shape[0] > 0:
        sesion = []
        for _ in range(6):
            mejor_toma = seleccion_voraz(solucion, sesion, tomas, data)
            sesion += [mejor_toma]
            idx = np.argwhere(tomas == mejor_toma)
            tomas = np.delete(tomas, idx)
        solucion.append(sesion)
        ses_cont += 1

    return solucion


sol = busqueda_voraz(TABLA_ESCENAS)
print("Costo: ", evaluar_solucion(sol, TABLA_ESCENAS) * 30)
print_schedule(np.array(sol).flatten())    


con coste:  840


Calendario de sesiones:
Orden de sesiones: [16, 27, 13, 28, 30, 25, 17, 19, 23, 14, 18, 24, 21, 5, 8, 9, 12, 22, 3, 15, 4, 11, 1, 6, 2, 20, 26, 7, 10, 29]
Horario:
 Día                  Tomas  Numero de actores  Coste de la sesion (€)
   1 16, 27, 13, 28, 30, 25                  5                     150
   2 17, 19, 23, 14, 18, 24                  3                      90
   3    21, 5, 8, 9, 12, 22                  6                     180
   4     3, 15, 4, 11, 1, 6                  7                     210
   5   2, 20, 26, 7, 10, 29                  7                     210

Coste Total:: 840€


# Mejor candidato
(a falta de implementar el resto de algoritmos)